## Cloning the required repository and navigating to the directory

In [1]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model
%cd first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 262, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 262 (delta 3), reused 5 (delta 1), pack-reused 253
Receiving objects: 100% (262/262), 72.13 MiB | 39.27 MiB/s, done.
Resolving deltas: 100% (130/130), done.
/content/first-order-model


##Mounting Gdrive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Making the display function

In [3]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")
 
def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))
 
    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])
 
    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

## model creation and loading of the checkpoints

In [4]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/First Order Model/vox-cpk.pth.tar')

## Choosing the image and driver path


In [7]:
#@title Set path for the video and image in your drive
face_path = "First Order Model/Emily.png" #@param {type:"string"}
driver_path = "First Order Model/drivers.mp4" #@param {type:"string"}

## Start predictions

In [8]:
from demo import make_animation
from skimage import img_as_ubyte
 
source_image = imageio.imread('/content/gdrive/My Drive/'+face_path)
reader = imageio.get_reader('/content/gdrive/My Drive/'+driver_path)
 
fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()
 
#Resize image and video to 256x256
 
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]
 
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                             adapt_movement_scale=True)
#save resulting video
imageio.mimsave('../output.mp4', [img_as_ubyte(frame) for frame in predictions])
 
HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 1079/1079 [00:42<00:00, 25.58it/s]


## Download cell

In [ ]:
from google.colab import files
files.download("/content/output.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>